In [12]:
import sys
import pandas as pd
from collections import defaultdict
from tqdm import tqdm

In [1]:
filepaths=!ls *.csv
filepaths=[i for i in filepaths if '@20' in i]
filepaths

['ADMM@20.csv',
 'EASE@20.csv',
 'FFM@20.csv',
 'GRU4RecF@20.csv',
 'ItemKNN@20.csv',
 'LGCN@20.csv',
 'NeuMF@20.csv',
 'Pop@20.csv',
 'SLIMElastic@20.csv']

In [33]:
filepaths=!ls *.csv
filepaths=[i for i in filepaths if '@20' in i]
filepaths

['ADMM@20.csv',
 'CDAE@20.csv',
 'EASE@20.csv',
 'FFM@20.csv',
 'GRU4RecF@20.csv',
 'ItemKNN@20.csv',
 'LGCN@20.csv',
 'NeuMF@20.csv',
 'Pop@20.csv',
 'RecVAE@20.csv',
 'SLIMElastic@20.csv']

In [7]:
filepaths.append('../EASE_20.csv')

In [8]:
ratios=[1,1,0.7,0.5,0.7,0.5,0.3,0.2,0.3,0.35]

In [10]:
w_dict = dict(zip(filepaths,ratios))
w_dict

{'ADMM@20.csv': 1,
 'EASE@20.csv': 1,
 'FFM@20.csv': 0.7,
 'GRU4RecF@20.csv': 0.5,
 'ItemKNN@20.csv': 0.7,
 'LGCN@20.csv': 0.5,
 'NeuMF@20.csv': 0.3,
 'Pop@20.csv': 0.2,
 'SLIMElastic@20.csv': 0.3,
 '../EASE_20.csv': 0.35}

In [32]:
print(rank_ratio)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3]


In [22]:
dataframe_list = []

print('순위별 가중치값 입력(ex: 1 0.9 0.8 ...)')
rank_ratio = [1 for _ in range(10)] + [0.3 for _ in range(5)]
rank_len = len(rank_ratio)

print(f"앙상블 모델 개수: {len(filepaths)}")

for i in range(len(filepaths)):
    dataframe_list.append(pd.read_csv(filepaths[i]))

user_list = dataframe_list[0]['user'].unique()
dataframe_len = len(dataframe_list)

순위별 가중치값 입력(ex: 1 0.9 0.8 ...)
앙상블 모델 개수: 10


In [23]:
len(user_list)

31360

In [24]:
ratios = list(w_dict.values())
K=10
result = []
tbar = tqdm(user_list, desc='Ensemble')
for user in tbar:
    temp = defaultdict(float)
    for df_idx in range(dataframe_len):
        items = dataframe_list[df_idx][dataframe_list[df_idx]['user'] == user]['item'].values
        max_rank = min(len(items), rank_len)
        for rank_idx in range(max_rank):
            temp[items[rank_idx]] += rank_ratio[rank_idx] * ratios[df_idx]

    for key, _ in sorted(temp.items(), key=lambda x: x[1], reverse=True)[:K]:
        result.append((user, key))

Ensemble: 100%|██████████████████████████████████████████████| 31360/31360 [06:37<00:00, 78.81it/s]


In [17]:
filename="--".join([f"{m.split('@')[0]}_{r}" for m,r in zip(filepaths[:-1],ratios[:-1]) if "@" in m])
filename

'ADMM_1--EASE_1--FFM_0.7--GRU4RecF_0.5--ItemKNN_0.7--LGCN_0.5--NeuMF_0.3--Pop_0.2--SLIMElastic_0.3'

In [30]:
submission = pd.DataFrame(result, columns=['user', 'item'])
submission.to_csv(f'{filename}.csv', index=False)

In [26]:
!ls

ADMM@20.csv				    ItemKNN@20.csv
EASE@10_1595.csv			    LGCN@20.csv
EASE@15.csv				    Make_Retrieval_Set.ipynb
EASE@20.csv				    NeuMF@10_1048.csv
EASE_1--FFM_0.6--NeuMF_0.6--RecVAE_0.9.csv  NeuMF@15.csv
EASE_1--FFM_0.7--NeuMF_0.7--RecVAE_1.csv    NeuMF@20.csv
EASE_1--FFM_1--NeuMF_1--RecVAE_1.csv	    Pop@15.csv
EASE_1--NeuMF_1--RecVAE_1.csv		    Pop@20.csv
Ensemble_1747.csv			    RecVAE@10_1279.csv
Ensemble_test.csv			    SLIMElastic@15.csv
Ensemble_test2.csv			    SLIMElastic@20.csv
FFM@10_1079.csv				    Slim@10.csv
FFM@15.csv				    cust_blend.ipynb
FFM@20.csv				    ensemble.ipynb
GRU4RecF@15.csv				    ensemble_k15.ipynb
GRU4RecF@20.csv				    ensemble_test.csv
ItemKNN@15.csv


In [15]:
!readlink -ef EASE_1--FFM_0.6--NeuMF_0.6--RecVAE_0.9.csv

/opt/ml/input/code/Ensemble/EASE_1--FFM_0.6--NeuMF_0.6--RecVAE_0.9.csv


In [26]:
file=pd.read_csv('SLIMElastic@15.csv')

In [27]:
users = file.groupby('user').user.head(10).reset_index(drop=True)
items = file.groupby('user').item.head(10).reset_index(drop=True)
file = pd.concat([users,items],axis=1)

In [29]:
file.to_csv("SLIMElastic@10.csv",index=False)

In [ ]:
ratios = []
dataframe_list = []

print('순위별 가중치값 입력(ex: 1 0.9 0.8 ...)')
rank_ratio = list(map(float, sys.stdin.readline().split()))
rank_len = len(rank_ratio)

num = int(input('앙상블할 모델 결과 개수: '))

for i in range(num):
    filepath = input(f'{i+1}번째 파일 경로: ')
    ratio = float(input(f'{i+1}번째 파일 가중치(0~1 사이 실수): '))
    dataframe_list.append(pd.read_csv(filepath))
    ratios.append(ratio)

user_list = dataframe_list[0]['user'].unique()
dataframe_len = len(dataframe_list)

result = []
tbar = tqdm(user_list, desc='Ensemble')
for user in tbar:
    temp = defaultdict(float)
    for df_idx in range(dataframe_len):
        items = dataframe_list[df_idx][dataframe_list[df_idx]['user'] == user]['item'].values
        max_rank = min(len(items), rank_len)
        for rank_idx in range(max_rank):
            temp[items[rank_idx]] += rank_ratio[rank_idx] * ratios[df_idx]

    for key, _ in sorted(temp.items(), key=lambda x: x[1], reverse=True)[:10]:
        result.append((user, key))

submission = pd.DataFrame(result, columns=['user', 'item'])
submission.to_csv(f'ensemble({num})-file.csv', index=False)